## Indexing

In [1]:
# all the boilerplate code we have in docs.ipynb is now also in the docs.py file

import docs

github_data = docs.read_github_data()
parsed_data = docs.parse_data(github_data)
chunks = docs.chunk_documents(parsed_data)

In [2]:
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")

In [3]:
dimensionality = 512
model_name = "jinaai/jina-embeddings-v2-small-en"
collection_name = "evidently-docs"

# qd_client.delete_collection(collection_name=collection_name)

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=dimensionality,
        distance=models.Distance.COSINE
    )
)

ResponseHandlingException: [Errno 111] Connection refused

In [ ]:
chunks[0]

{'start': 0,
 'content': '<Note>\n  If you\'re not looking to build API reference documentation, you can delete\n  this section by removing the api-reference folder.\n</Note>\n\n## Welcome\n\nThere are two ways to build API documentation: [OpenAPI](https://mintlify.com/docs/api-playground/openapi/setup) and [MDX components](https://mintlify.com/docs/api-playground/mdx/configuration). For the starter kit, we are using the following OpenAPI specification.\n\n<Card\n  title="Plant Store Endpoints"\n  icon="leaf"\n  href="https://github.com/mintlify/starter/blob/main/api-reference/openapi.json"\n>\n  View the OpenAPI specification file\n</Card>\n\n## Authentication\n\nAll API endpoints are authenticated using Bearer tokens and picked up from the specification file.\n\n```json\n"security": [\n  {\n    "bearerAuth": []\n  }\n]\n```',
 'title': 'Introduction',
 'description': 'Example section for showcasing API endpoints',
 'filename': 'api-reference/introduction.mdx'}

In [ ]:
points = []

for i, doc in enumerate(chunks):
    text = doc.get('title', '') + ' ' + doc.get('description', '') + ' ' + doc['content']
    text = text.strip()
    vector = models.Document(text=text, model=model_name)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [ ]:
len(points)

575

In [ ]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

## Retrieval

In [ ]:
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")

model_name = "jinaai/jina-embeddings-v2-small-en"
collection_name = "evidently-docs"

In [ ]:
def vector_search(query, num_results=15):    
    vector = models.Document(text=query, model=model_name)

    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=vector,
        limit=num_results,
        with_payload=True
    )

    results = []

    for point in query_points.points:
        results.append(point.payload)

    return results

In [ ]:
search_results = vector_search('how do I use llm-as-a-judge for evals')

In [ ]:
search_results[0]

{'start': 20000,
 'content': 'openai",\n            model = "gpt-4o-mini",\n            alias="Verbosity")\n    ])\n```\n\nRun the Report and view the summary results:\xa0\n\n```python\nreport = Report([\n    TextEvals()\n])\n\nmy_eval = report.run(eval_dataset, None)\nmy_eval\n```\n\n![](/images/examples/llm_judge_tutorial_verbosity-min.png)\n\nYou can also view the dataframe using `eval_dataset.as_dataframe()`\n\n<Info>\n  Don\'t fully agree with the results? Use these labels as a starting point, edit the decisions where you see fit - now you\'ve got your golden dataset\\! Next, iterate on your judge prompt. You can also try different evaluator LLMs to see which one does the job better. [How to change an LLM](/metrics/customize_llm_judge#change-the-evaluator-llm).\n</Info>\n\n## What\'s next?\n\nThe LLM judge itself is just one part of your overall evaluation framework. You can integrate this evaluator into different workflows, such as testing your LLM outputs after changing a prompt

In [ ]:
def rag(question):
    search_results = vector_search(question)
    ...
    